In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import ta
import matplotlib.pyplot as plt
import vnstock as vn
from datetime import timedelta
from vnstock3 import Vnstock

# Setting

In [3]:
rf = {'2019': 0.0451, '2020': 0.0286, '2021':0.023,'2022': 0.0335,'2023':0.0321}
rf = pd.Series(rf)
rf_cal = rf.mean()
rf_2022 = 0.0335
rf_cal

0.03246

In [4]:
pd.set_option('display.max_columns', None) 

In [5]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_SLOW_PERIOD = 26
MACD_FAST_PERIOD = 12
MACD_SIGNAL_PERIOD = 9
initial_investment = 100_000_000
# backup_amount_initial = 40_000_000

In [6]:
win_rate = 0.5711907018
loss_rate = 1 - win_rate
mean_profit = 0.3493357553
mean_loss = 0.1397008027
stop_loss = 0.08

In [7]:
def kelly_criterion(p, q, profit, loss):
    b = (profit * 160_000_000)/(loss * 160_000_000)
    f = (b*p - q)/b
    return f

In [8]:
f =kelly_criterion(win_rate,loss_rate,mean_profit,mean_loss)
f

0.39970810302530857

In [9]:
high_ESG_group = ['CTD', 'DHG', 'DPM', 'FPT', 'GAS', 'MBB']
low_ESG_group = ['NVL', 'PNJ', 'REE', 'SBT', 'SSI', 'STB', 'VIC', 'VNM']
non_ESG_group = ['CII', 'CTG', 'EIB', 'GMD', 'HDB', 'HPG', 'MSN', 'MWG', 'ROS', 'SAB', 'TCB', 'VCB', 'VHM', 'VJC', 'VPB', 'VRE']


In [10]:
companies = ['VCB']

# Calculate indicators

In [11]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [12]:
def macd_strategy(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
        (df['MACD'] >= df['Signal_Line']) &
        (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    # Sell Signals: 
    df.loc[
        (df['RSI'] < RSI_OVERBOUGHT) &
        (df['Previous_MACD'] > df['Previous_Signal_Line']) &
        (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [13]:
def get_next_trading_day(date, trading_days):
    while date not in trading_days:
        date += BDay(1)
    return date

In [14]:
def calculate_daily_risk_free_rate(annual_rate_decimal, trading_days=252):
    # Convert percentage to decimal
    # Calculate daily risk-free rate using the compound formula
    daily_rate = (1 + annual_rate_decimal) ** (1 / trading_days) - 1
    
    # Alternatively, for a simple calculation:
    # daily_rate = annual_rate_decimal / trading_days
    
    return daily_rate

# Example usage
annual_yield = 0.0326  # Example annual yield of 3%
daily_risk_free_rate = calculate_daily_risk_free_rate(rf_cal)
daily_risk_free_rate_2022 = calculate_daily_risk_free_rate(rf_2022)


# Backtest

In [21]:
def simulate_investment(
    ticker, win_rate, loss_rate, mean_profit, mean_loss, 
    sell_fraction, start_date, end_date, f_star=1
):
    try:
        # Initialize trade counters and portfolio metrics
        number_of_buying_trades = 0
        number_of_selling_trades = 0
        cash = initial_investment
        holdings = 0
        portfolio_values = []

        # Load stock data and calculate indicators
        data = vn.stock_historical_data(ticker, start_date, end_date, resolution='1D', type='stock', source='TCBS')
        data = data.set_index(pd.DatetimeIndex(data['time'].values))
        data = calculate_indicators(data)
        data = macd_strategy(data)

        trading_days = data.index
        buy_signals = data[data['Signal'] == 1].index
        sell_signals = data[data['Signal'] == -1].index

        pending_buy_shares = {}
        pending_sell_revenue = {}
        
        for i, current_date in enumerate(data.index):
            current_price = data['close'].iloc[i]

            # Handle pending T+2 settlements
            if current_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(current_date)
            if current_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(current_date)

            # Avoid trades in January 2024
            if current_date.month == 1 and current_date.year == 2024:
                portfolio_values.append(cash + holdings * current_price)
                continue

            # Buy if there's a buy signal and cash allows
            if current_date in buy_signals:
                allocation = cash * f_star
                shares_to_buy = int(allocation // current_price)
                total_cost = shares_to_buy * current_price
                if shares_to_buy > 0 and cash >= total_cost:
                    cash -= total_cost
                    settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                    pending_buy_shares[settlement_date] = pending_buy_shares.get(settlement_date, 0) + shares_to_buy
                    last_buy_price = current_price
                    number_of_buying_trades += 1

            # Sell if there's a sell signal and holdings allow
            if holdings > 0 and current_date in sell_signals:
                shares_to_sell = int(holdings * sell_fraction)
                revenue = shares_to_sell * current_price
                holdings -= shares_to_sell
                settlement_date = get_next_trading_day(current_date + BDay(2), trading_days)
                pending_sell_revenue[settlement_date] = pending_sell_revenue.get(settlement_date, 0) + revenue
                number_of_selling_trades += 1

            # Update portfolio value
            portfolio_values.append(cash + holdings * current_price)

        # Finalize portfolio values including pending settlements
        final_date = data.index[-1]
        while final_date <= data.index[-1] + BDay(2):
            if final_date in pending_buy_shares:
                holdings += pending_buy_shares.pop(final_date)
            if final_date in pending_sell_revenue:
                cash += pending_sell_revenue.pop(final_date)
            portfolio_values.append(cash + holdings * data['close'].iloc[-1])
            final_date += BDay(1)

        # Adjust portfolio values to match data index length
        if len(portfolio_values) > len(data.index):
            portfolio_values = portfolio_values[:len(data.index)]
        elif len(portfolio_values) < len(data.index):
            portfolio_values.extend([portfolio_values[-1]] * (len(data.index) - len(portfolio_values)))

        # Add portfolio values to data frame
        data['Portfolio_Value'] = portfolio_values
        data['value'] = portfolio_values
        data['Number_of_Buying_Trades'] = number_of_buying_trades
        data['Number_of_Selling_Trades'] = number_of_selling_trades
        # Ensure the column 'value' is directly modified in the original DataFrame
        data['value'] = data['value'].apply(lambda x: np.nan if pd.notna(x) and x < 1000000 else x)
        data['value'] = data['value'].ffill()

        data['Daily_Return'] = data['value'].pct_change()
        data['Accumulated_Profit'] = data['value'] - initial_investment

        data['Running_Max'] = data['value'].cummax()  # Track the running max portfolio value
        data['Drawdown'] = (data['value'] - data['Running_Max']) / data['Running_Max']  # Calculate drawdown
        data = data.dropna(subset=['time'])
        data = data.drop_duplicates(subset=['time', 'open', 'high', 'low', 'close', 'volume'], keep='first')
        return data
        

    except Exception as e:
        print(f"Error occurred for {ticker}: {e}")
        return pd.DataFrame()


In [16]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return and standard deviation of returns
    average_return = daily_returns.mean()
    std_deviation = daily_returns.std()

    # Calculate the Sharpe Ratio
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation if std_deviation > 0 else np.nan

    return sharpe_ratio

In [17]:
def calculate_sortino_ratio(data,rf=0.01):  # Target return can be set to risk-free rate
    # Calculate daily returns from the Portfolio Value
    daily_returns = data['Daily_Return'].dropna()

    # Calculate average return
    average_return = daily_returns.mean()

    # Calculate downside returns (returns below the target return)
    downside_returns = daily_returns[daily_returns < rf]

    # Calculate downside deviation
    downside_deviation = downside_returns.std() if not downside_returns.empty else np.nan

    # Calculate the Sortino Ratio
    sortino_ratio = (average_return - rf) / downside_deviation if downside_deviation > 0 else np.nan

    return sortino_ratio

thêm phần điều chỉnh kelly trong code phía dưới

In [36]:
def backtest_multiple_companies(companies_vn30, win_rate, loss_rate, mean_profit, mean_loss, \
                                sell_fraction, start_date, end_date, rf):
    results = []
    for company in companies_vn30:
        result = simulate_investment(company, win_rate, loss_rate, mean_profit, mean_loss, \
                                     sell_fraction, start_date=start_date, end_date=end_date, f_star=1)
        if not result.empty:
            # Calculate the Sharpe Ratio for the result
            sharpe_ratio = calculate_sharpe_ratio(result,risk_free_rate=rf)
            results.append({
                'Company': company,
                'Final Portfolio Value': result['Portfolio_Value'].iloc[-1],
                'Total Profit': result['Accumulated_Profit'].iloc[-1],
                'Rate of Return': result['Accumulated_Profit'].iloc[-1] / initial_investment * 100,
                'Number of Buying Trades': result['Number_of_Buying_Trades'].max(),
                'Number of Selling Trades': result['Number_of_Selling_Trades'].max(),
                'Sharpe Ratio': sharpe_ratio,
                'Sortino Ratio': calculate_sortino_ratio(result,rf=rf),
                'MDD': result['Drawdown'].min(),
            })
    return pd.DataFrame(results)

# Kelly 2019-2024

In [19]:
sell_fraction = 1

In [38]:
 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
results_df = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                        sell_fraction, start_date='2018-12-28' ,end_date='2024-01-05', rf=daily_risk_free_rate)
high_esg = pd.DataFrame(results_df)
print(high_esg)
average_rate_of_return = results_df['Rate of Return'].mean()
average_profit = results_df[results_df['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = results_df[results_df['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}','avg sharpe:',results_df['Sharpe Ratio'].mean(),'avg sortino:',results_df['Sortino Ratio'].mean())

Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
Time range is 1834 days. Looping through 6 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     CTD              246967757   146967757.0      146.967757   
1     DHG              123524130    23524130.0       23.524130   
2     DPM              260732160   160732160.0      160.732160   
3     FPT              255201564   155201564.0      155.201564   
4     GAS              207624920   107624920.0      107.624920   
5     MBB              193101590    93101590.0       93.101590   

   Number of Buying Trades  Number of Selling Trades  Sharpe Ratio  \
0                       35                        34      0.042659   
1                       38                        37      0.009250   
2              

In [39]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss,\
                                             sell_fraction, start_date='2018-12-29', end_date='2024-01-05', \
                                                rf=daily_risk_free_rate)
low_esg_kelly = pd.DataFrame(low_esg_kelly)
print(low_esg_kelly)
average_rate_of_return = low_esg_kelly['Rate of Return'].mean()
average_profit = low_esg_kelly[low_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly[low_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
E

In [40]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2018-12-29', end_date='2024-01-05', rf=daily_risk_free_rate)
non_esg_kelly = pd.DataFrame(non_esg_kelly)
print(non_esg_kelly)
average_rate_of_return = non_esg_kelly['Rate of Return'].mean()
average_profit = non_esg_kelly[non_esg_kelly['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly[non_esg_kelly['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')

Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Time range is 1833 days. Looping through 6 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
Error 429. {
  "message":"API rate limit exceeded"
}
E

In [41]:
high_esg['ESG'] ='High'
low_esg_kelly['ESG'] = 'Low'
non_esg_kelly['ESG'] = 'Non'

In [42]:
merged = pd.concat([high_esg, low_esg_kelly, non_esg_kelly]) 
merged['period'] = '2019-2024' 

# KELLY 2022

In [43]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
high_esg_2022 = backtest_multiple_companies(high_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
high_esg_2022 = pd.DataFrame(high_esg_2022)
print(high_esg_2022)
average_rate_of_return = high_esg_2022['Rate of Return'].mean()
average_profit = high_esg_2022[high_esg_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = high_esg_2022[high_esg_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
high_esg_2022['ESG'] ='High'

Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for CTD: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for DHG: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for DPM: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for FPT: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for GAS: 'NoneType' object is not subscriptable
Time range is 365 days. Looping through 2 requests
Error 429. {
  "message":"API rate limit exceeded"
}
Error occurred for MBB: 'NoneType' object is not subscriptabl

KeyError: 'Rate of Return'

In [ ]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
low_esg_kelly_2022 = backtest_multiple_companies(low_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
low_esg_kelly_2022 = pd.DataFrame(low_esg_kelly_2022)
print(low_esg_kelly_2022)
average_rate_of_return = low_esg_kelly_2022['Rate of Return'].mean()
average_profit = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = low_esg_kelly_2022[low_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
low_esg_kelly_2022['ESG'] = 'Low'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Error occurred for PNJ: Out of bounds nanosecond timestamp: 9223545600000000000
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
  Company  Final Portfolio Value  Total Profit  Rate of Return  \
0     NVL              103285500       3285500        3.285500   
1     REE               96529470      -3470530       -3.470530   
2     SBT              104998160       4998160        4.998160   
3     SSI               97988360      -2011640       -2.011640   
4     STB              106453756       6453756        6.453756   
5     VIC               98264100      -1735900       -1.735900   
6     VNM              100251120        251120    

In [ ]:
sell_fraction = 1 #kelly_criterion(win_rate, loss_rate, mean_profit, mean_loss)
non_esg_kelly_2022 = backtest_multiple_companies(non_ESG_group, win_rate, loss_rate, mean_profit, mean_loss, \
                                            sell_fraction, start_date='2022-01-01', end_date='2023-01-01', \
                                                rf=daily_risk_free_rate_2022)
non_esg_kelly_2022 = pd.DataFrame(non_esg_kelly_2022)
print(non_esg_kelly_2022)
average_rate_of_return = non_esg_kelly_2022['Rate of Return'].mean()
average_profit = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] > 0]['Rate of Return'].mean()
average_loss = non_esg_kelly_2022[non_esg_kelly_2022['Rate of Return'] < 0]['Rate of Return'].mean()
print("Average Rate of Return for 30 companies:", average_rate_of_return)
print(f'avg_ror: {average_rate_of_return}, avg_profit: {average_profit}, avg loss: {average_loss}')
non_esg_kelly_2022['ESG'] = 'Non'

Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Error occurred for SAB: Out of bounds nanosecond timestamp: 9223545600000000000
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Time range is 365 days. Looping through 2 requests
Error occurred for VPB: Out of bounds nanosecond timestamp: 9223545600000000000
Time range is 365 days. Looping through 2 requests
   Company  Final Portfo

In [ ]:
merged_2022 = pd.concat([high_esg_2022, low_esg_kelly_2022, non_esg_kelly_2022])
merged_2022['period'] = '2022-2023'
merged_2022

,Company,Final Portfolio Value,Total Profit,Rate of Return,Number of Buying Trades,Number of Selling Trades,Sharpe Ratio,Sortino Ratio,MDD,ESG,period
0,CTD,100810550,810550,0.810550,8,8,0.084288,0.148321,-0.471670,High,2022-2023
1,DHG,93147170,-6852830,-6.852830,8,7,0.080253,0.158319,-0.662976,High,2022-2023
2,DPM,95260550,-4739450,-4.739450,8,8,0.080460,0.145286,-0.466581,High,2022-2023
3,FPT,108893350,8893350,8.893350,5,5,0.071708,0.126683,-0.448794,High,2022-2023
4,GAS,104567690,4567690,4.567690,9,7,0.089827,0.176773,-0.658464,High,2022-2023
5,MBB,101609360,1609360,1.609360,6,6,0.072588,0.126856,-0.444657,High,2022-2023
0,NVL,103285500,3285500,3.285500,7,6,0.079202,0.158002,-0.650548,Low,2022-2023
1,REE,96529470,-3470530,-3.470530,9,9,0.086395,0.151266,-0.460738,Low,2022-2023
2,SBT,104998160,4998160,4.998160,8,7,0.084421,0.149111,-0.470806,Low,2022-2023
3,SSI,97988360,-2011640,-2.011640,7,7,0.081271,0.144670,-0.482601,Low,2022-2023


# gop 2 cai

In [ ]:
merged_all = pd.concat([merged, merged_2022])
merged_all= merged_all.reset_index()
merged_all['total_trades']= merged_all['Number of Buying Trades'] + merged_all['Number of Selling Trades']
merged_all.to_csv('macd no kelly.csv', index=False)

In [23]:
a = simulate_investment('NVL', win_rate, loss_rate, mean_profit, mean_loss, \
                                     sell_fraction, start_date='2019-01-02', end_date='2023-01-02', f_star=1)
a

Time range is 1461 days. Looping through 5 requests


,time,open,high,low,close,volume,ticker,RSI,Previous_RSI,MACD,Signal_Line,Previous_MACD,Previous_Signal_Line,Signal,Portfolio_Value,value,Number_of_Buying_Trades,Number_of_Selling_Trades,Daily_Return,Accumulated_Profit,Running_Max,Drawdown
2019-01-08,2019-01-08,34500,34500,32100,32100,621630,NVL,NaN,0.000000,NaN,NaN,0.000000,0.000000,0,100000000,100000000.0,35,35,NaN,0.0,100000000.0,0.000000
2019-01-09,2019-01-09,32159,33380,29880,32720,1165350,NVL,NaN,0.000000,NaN,NaN,0.000000,0.000000,0,100000000,100000000.0,35,35,0.0,0.0,100000000.0,0.000000
2019-01-10,2019-01-10,32270,33270,32270,32940,470210,NVL,NaN,0.000000,NaN,NaN,0.000000,0.000000,0,100000000,100000000.0,35,35,0.0,0.0,100000000.0,0.000000
2019-01-11,2019-01-11,32549,33270,32549,32990,627520,NVL,NaN,0.000000,NaN,NaN,0.000000,0.000000,0,100000000,100000000.0,35,35,0.0,0.0,100000000.0,0.000000
2019-01-14,2019-01-14,32549,32659,31660,32330,717640,NVL,NaN,0.000000,NaN,NaN,0.000000,0.000000,0,100000000,100000000.0,35,35,0.0,0.0,100000000.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,2022-12-26,15200,15200,14050,14050,12071115,NVL,15.699067,16.882585,-6047.756267,-7133.438653,-6228.846185,-7404.859249,0,179663970,179663970.0,35,35,0.0,79663970.0,190498670.0,-0.056875
2022-12-27,2022-12-27,13200,14400,13100,14400,27214010,NVL,17.768436,15.699067,-5809.036174,-6868.558157,-6047.756267,-7133.438653,0,179663970,179663970.0,35,35,0.0,79663970.0,190498670.0,-0.056875
2022-12-28,2022-12-28,14500,14600,13800,14350,13459048,NVL,17.701585,17.768436,-5559.793524,-6606.805231,-5809.036174,-6868.558157,0,179663970,179663970.0,35,35,0.0,79663970.0,190498670.0,-0.056875
2022-12-29,2022-12-29,14050,15350,14050,14650,21575450,NVL,19.654804,17.701585,-5277.226849,-6340.889554,-5559.793524,-6606.805231,0,179663970,179663970.0,35,35,0.0,79663970.0,190498670.0,-0.056875


In [ ]:
# kelly = pd.read_csv('macd kelly.csv')
# no_kelly = pd.read_csv('macd no kelly.csv')
# kelly['Kelly'] = 'Yes'
# no_kelly['Kelly'] = 'No'
# double_merged = pd.concat([kelly, no_kelly])
# double_merged.to_excel('macd.xlsx', index=False)